# Feature Engineering and Data Preparation

FE is an important step in data science process. Sometimes this process is iterative and it is not straight forward, we can define models and then go back to EDA and then create a new feature, so it is imnportant to modularize as much as possible your data preparation steps. This is why I will build this notebook, I will use sklearn pipelines so we can process data and create features discussed in previous notebook.

So the major is to finish data preparation so we can train models with our data. Folowwing steps will be applied:
 - Label Encode categorical variables.
 - ColumnCapping for self-reported variables.
 - Missing Imputation for missing values.
 - Remove columns that has leakage. (ex: recoveries)

I will postpone a few preprocess steps because they will be defined together with models, such as OneHotEncoder (necessary for a few models only), Data Scaling and train / test splits. All of this will be done in next notebook (Model Training and Result Analysis).

## Importing libraries and data

In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import sys
import logging

sys.path.append('../src/')

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline

from sklego.preprocessing import ColumnCapper

# custom functions and classes
from features import NewFeatureTransformer
import preprocess

pd.set_option("max_columns", 50)

In [23]:
df = pd.read_csv("../data/raw/loan.csv")
df.shape

(2260668, 14)

In [24]:
columns_to_drop = ['recoveries', 'last_pymnt_d', 'loan_status', 'earliest_cr_line']

In [25]:
df_loan_filtered = (
    df
     .pipe(preprocess.initialize_pipeline)
     .pipe(preprocess.remove_trailing_spaces, "term")
     .pipe(preprocess.correct_dtypes, ["term"], [np.int])
     .pipe(preprocess.filter_not_default_or_paid_loans)
     .pipe(preprocess.convert_to_datetime)
     .pipe(preprocess.binarize_loan_status)
     .pipe(preprocess.calculate_credit_time_years)
     .pipe(preprocess.filter_columns, columns_to_drop)
)

DEBUG:preprocess:[initialize_pipeline(df)] n_obs=2260668 n_col=14 time=0:00:00.198794
DEBUG:preprocess:[remove_trailing_spaces(df, column = 'term')] n_obs=2260668 n_col=14 time=0:00:04.480236
INFO:preprocess:[filter_not_default_or_paid_loans(df)] n_obs=1328284 n_col=14 time=0:00:00.302517
INFO:preprocess:[binarize_loan_status(df)] n_obs=1328284 n_col=15 time=0:00:00.144662
INFO:preprocess:[calculate_credit_time_years(df)] n_obs=1328284 n_col=16 time=0:00:00.044704


Now that we did all steps for cleaning our data using pandas pipelines, I will use a more formal approach using sklearn Pipelines. This is important because defining these steps will separate all transformations applied to train and test data avoiding leakage. For example, I discussed that annual_inc is a biased variable and since it is self-reported, we need to apply column capping, then to compute other features based on annual_inc I need first to apply capping then calculate features. Moreover, this will ease my work when I use cross-validation in k-fold because I will not worry about these leakages between folders.

In [26]:
df_loan_filtered.head(10)

,term,int_rate,installment,grade,sub_grade,home_ownership,annual_inc,issue_d,dti,application_type,loan_status_general,credit_time_in_years
100,36,22.35,1151.16,D,D5,MORTGAGE,100000.0,2018-12-01,30.46,Joint App,fully_paid,6.92
152,60,16.14,975.71,C,C4,MORTGAGE,45000.0,2018-12-01,50.53,Joint App,fully_paid,9.50
170,36,7.56,622.68,A,A3,MORTGAGE,100000.0,2018-12-01,18.92,Joint App,fully_paid,19.83
186,36,11.31,147.99,B,B3,RENT,38500.0,2018-12-01,4.64,Individual,fully_paid,15.00
215,36,27.27,345.18,E,E5,MORTGAGE,450000.0,2018-12-01,12.37,Joint App,fully_paid,21.17
269,60,17.97,507.55,D,D1,RENT,57000.0,2018-12-01,22.18,Individual,fully_paid,23.67
271,36,11.31,217.05,B,B3,RENT,45000.0,2018-12-01,16.21,Individual,fully_paid,9.67
296,36,13.56,84.92,C,C1,RENT,42000.0,2018-12-01,15.09,Individual,fully_paid,14.75
369,36,17.97,144.55,D,D1,MORTGAGE,60000.0,2018-12-01,19.10,Individual,fully_paid,11.50
379,36,8.19,84.85,A,A4,OWN,24000.0,2018-12-01,24.95,Individual,fully_paid,16.17


In [27]:
ft = NewFeatureTransformer()
cc = ColumnCapper()
imputer = SimpleImputer(strategy='median')

preprocess_ct = ColumnTransformer([
     ("new_features", ft, ["annual_inc", "int_rate", "term", "installment"]),
     ("capping", cc, ["dti", "credit_time_in_years"]),
     ("imputer", imputer, ["dti", "annual_inc", "credit_time_in_years"])
], remainder='drop')

pd.DataFrame(ft.fit_transform(df_loan_filtered)).head(10)

,term,int_rate,installment,grade,sub_grade,home_ownership,annual_inc,issue_d,dti,application_type,loan_status_general,credit_time_in_years,total_loan_amount,installment_to_income,total_loan_to_income
100,36,22.35,1151.16,D,D5,MORTGAGE,100000.0,2018-12-01,30.46,Joint App,fully_paid,6.92,2108.373804,13.813920,0.702791
152,60,16.14,975.71,C,C4,MORTGAGE,45000.0,2018-12-01,50.53,Joint App,fully_paid,9.50,2061.720858,26.018933,0.916320
170,36,7.56,622.68,A,A3,MORTGAGE,100000.0,2018-12-01,18.92,Joint App,fully_paid,19.83,774.849393,7.472160,0.258283
186,36,11.31,147.99,B,B3,RENT,38500.0,2018-12-01,4.64,Individual,fully_paid,15.00,204.096200,4.612675,0.176707
215,36,27.27,345.18,E,E5,MORTGAGE,155000.0,2018-12-01,12.37,Joint App,fully_paid,21.17,711.580039,2.672361,0.153028
269,60,17.97,507.55,D,D1,RENT,57000.0,2018-12-01,22.18,Individual,fully_paid,23.67,1159.676160,10.685263,0.406904
271,36,11.31,217.05,B,B3,RENT,45000.0,2018-12-01,16.21,Individual,fully_paid,9.67,299.338335,5.788000,0.221732
296,36,13.56,84.92,C,C1,RENT,42000.0,2018-12-01,15.09,Individual,fully_paid,14.75,124.361553,2.426286,0.098700
369,36,17.97,144.55,D,D1,MORTGAGE,60000.0,2018-12-01,19.10,Individual,fully_paid,11.50,237.319177,2.891000,0.131844
379,36,8.19,84.85,A,A4,OWN,28000.0,2018-12-01,24.95,Individual,fully_paid,16.17,107.451680,3.636429,0.127919


Above I built a transformer that calculates new features, note that it is important the usage of transformers to isolate train / test sets. Also, missing values are too litle and a simple imputer strategy is enough for this case.

In [28]:
df_loan_filtered = ft.fit_transform(df_loan_filtered)
df_loan_filtered.isnull().sum()

term                       0
int_rate                   0
installment                0
grade                      0
sub_grade                  0
home_ownership             0
annual_inc                 4
issue_d                    0
dti                      340
application_type           0
loan_status_general        0
credit_time_in_years      29
total_loan_amount          0
installment_to_income      4
total_loan_to_income       4
dtype: int64

Now let's save filtered data and work in next notebook where you are going to build and analyze machine learning models for task proposed.

In [30]:
df_loan_filtered.to_csv("../data/preprocessed/df_loan_filtered.csv", index=False)